In [1]:
import requests
import json
test_url = "http://knowledge-engine.easygov.co.in:1880/getInferenceGraphData?file_name=Test_weightage_InferenceGraph.json"
prod_url = "http://knowledge-engine.easygov.co.in:1880/getInferenceGraphData?file_name=Demo%20Project_InferenceGraph.json"
response = requests.get(prod_url)
res = json.loads(response.text)
links = res['links']
ILINES = res['ILINES']
nodes = res['nodes']
# keys = None
# for node in nodes:
#     if not keys:
#         keys = list(node.keys())
#     for key in keys:
#         if key not in ['id', 'PTABLE', 'CPT']:
#             node.pop(key)  
t = set()
for link in links:
    t.add(link['target'])
targets = list(t)
source = list()
all_links = []
data = dict()
for target in targets:
    for link in links:
        if target == link['target']:
            source.append(link['source'])
    data.update({target:list(set(source))})
    all_links.append(data)
    data = dict()
    source = list()
percentage_contribution = dict()
for target in targets:
    pcs = list()
    ratios = list()
    percentages = list(range(1))
    low_value = ""
    for node in nodes:
        if node['id']==target:
            PTABLE = node['PTABLE']
            CPT = node['CPT']
            table = list()
            for tab in PTABLE:
                table.append(tab["data"])
            total= 0
            for row in CPT:
                total += float(row[0])
            parent_nodes = None
            for link in all_links:
                t = list(link.keys())[0]
                if t.lower()==target.lower():
                    parent_nodes = link[target]
            if parent_nodes:
                l = len(parent_nodes)
                for i in range(l):
                    low_value = PTABLE[0]["data"][i].lower()
                    factual_contri = 0
                    for row in table:
                        factual_contri += float(row[l]) if row[i].lower() == low_value else 0
                    proportional_contribution = abs(2*factual_contri - total)/total
                    pcs.append(proportional_contribution)
                ratios = [pc/pcs[0] for pc in pcs]
                percentages[0] = 100/sum(ratios)
                percentages = [round(percentages[0]*r, 2) for r in ratios]
                data = dict()
                for i in range(l):
                    data.update({f"{parent_nodes[i]}[{low_value}]":percentages[i]})
                percentage_contribution.update({target:data})
# Data in Json Format              
percentage_contribution_json = json.dumps(percentage_contribution)

In [3]:
percentage_contribution_json

'{"1to5": {"1to5SchoolType[madarsa]": 45.2, "PrimaryMerit[madarsa]": 2.02, "No1to5[madarsa]": 52.79}, "FamilyInfo": {"FamilyComposition[low]": 67.62, "HealthDependents[low]": 29.72, "SocialProfile[low]": 2.65}, "AdolescentGirlNutrition": {"AdolescentGirls[low]": 45.49, "FoodBenefit[low]": 54.51}, "AdvancedAmenities": {"CookingFacility[none]": 62.61, "WasteWaterOutlet[none]": 37.39}, "Primary": {"1to5[low]": 9.56, "6to8[low]": 90.44}, "Taxable": {"Taxable_FamilyIncome[low]": 57.93, "TaxSlab[low]": 42.07}, "HouseConstruction": {"HouseBenefit[none]": 16.68, "HouseValue[none]": 83.32}, "MedicalDevices": {"HealthBenefit[low]": 40.49, "Disability[low]": 59.51}, "LoanOrGrant": {"Graduation[low]": 22.82, "EducationBenefit[low]": 18.24, "EconomicProfile[low]": 58.94}, "NonAgriLand": {"ResidentialProperty[low]": 35.71, "CulturableWasteLand[low]": 41.56, "CommercialProperty[low]": 22.73}, "UnskilledWork": {"OccupationBenefit[adhoc]": 43.53, "IncomeType_NonTaxable[adhoc]": 27.98, "EarningMembersNo